<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1a_OLCI_data_access_Data_Store.ipynb" target="_blank"><< Accessing OLCI data</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_3_OLCI_coverage.ipynb" target="_blank">Determining OLCI product coverage >></a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2023 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h3>Learn OLCI: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
The following modules are prerequisites for this notebook, and will retrieve the data required here.
  - **<a href="./1_1a_OLCI_data_access_Data_Store.ipynb" target="_blank">1_1a_OLCI_data_access_Data_Store.ipynb</a>** if using the Data Store for data access
    <br><br>**OR**<br><br>
  - **<a href="./1_1b_OLCI_data_access_HDA.ipynb" target="_blank">1_1b_OLCI_data_access_HDA.ipynb</a>** if using WEkEO for data access
    
</div>
<hr>

# 1.2 Understanding OLCI product structure

### Data used

| Product Description | Data Store collection ID| Product Navigator | WEkEO HDA ID | WEkEO metadata |
|:--------------------:|:-----------------------:|:-------------:|:-----------------:|:-----------------:|
| Sentinel-3 OLCI level-1B Full resolution | EO:EUM:DAT:0409 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:OL_1_EFR___NTC?query=OLCI&filter=satellite__Sentinel-3&filter=instrument__OLCI&filter=processingLevel__Level%201%20Data&s=advanced" target="_blank">link</a>| EO:EUM:DAT:SENTINEL-3:OL_1_EFR___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3AOL_1_EFR___" target="_blank">link</a> |
| Sentinel-3 OLCI level-2 full resolution | EO:EUM:DAT:0407 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:OL_2_WFR___NTC?query=OLCI&filter=satellite__Sentinel-3&filter=instrument__OLCI&filter=processingLevel__Level%202%20Data&s=advanced" target="_blank">link</a> | EO:EUM:DAT:SENTINEL-3:OL_2_WFR___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3AOL_1_EFR___" target="_blank">link</a> |

### Learning outcomes

At the end of this notebook you will know;
* What the SAFE format is
* What components are inside a SAFE format file from OLCI
* What variables are in present in each component for OLCI data files (level-1B and level-2)

### Outline 

Data from all the Sentinel satellites operated under the European Commissions Copernicus Programme are delievered in "**SAFE format**". The Sentinel-SAFE format is a specific variation of the Standard Archive Format for Europe (SAFE) format specification designed for the Sentinel satellite products.  It is based on the XML Formatted Data Units (XFDU) standard under development by the Consultative Committee for Space Data Systems (CCSDS). Sentinel-SAFE is a profile of XFDU, and it restricts the XFDU specifications for specific utilisation in the Earth Observation domain, providing semantics in the same domain to improve interoperability between ground segment facilities.


Each product package includes:

* a manifest file containing a metadata section and a data object section (an xml file).

* measurement data files (NetCDF-4 format)

* annotation data files, if defined (NetCDF-4 format)

The product package can exist as a directory in a filesystem, zipped folder or tarball.

The naming of the Sentinel-SAFE files follows a specific convention that you can learn about <a href="https://eumetsatspace.atlassian.net/wiki/spaces/DPF/pages/edit-v2/1597702263?" target="_blank">here</a>

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
 1. [Querying OLCI file structure (Level-1B)](#section1)
 2. [Querying OLCI file structure (Level-2)](#section2)
 3. [Applying your knowledge](#section3)

<hr>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [1]:
# library imports
import os                                     # a library that allows us access to basic operating system commands
import glob                                   # a library that aids in searching for files
from IPython.display import display, Markdown # a library that helps us display HTML and markdown
import xarray as xr                           # a library that supports the use of multi-dimensional arrays in Python
import xml.etree.ElementTree as ET            # a library that helps us parse XML files
import warnings                               # a library that helps us manage warnings
warnings.filterwarnings('ignore')

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Querying OLCI file structure (Level-1B)
[Back to top](#TOC_TOP)

</div>

First we will create a variable which holds the path for the file we are interested in, in this case the level-1B OLCI file that we downloaded in either <a href="./1_1a_OLCI_data_access_Data_Store.ipynb" target="_blank">1_1a_OLCI_data_access_Data_Store.ipynb</a> or <a href="./1_1b_OLCI_data_access_HDA.ipynb" target="_blank">1_1b_OLCI_data_access_HDA.ipynb</a>.

In [2]:
# selecting SAFE directory
SAFE_directory = os.path.join(os.getcwd(), 'products', 
    'S3A_OL_1_EFR____20210717T101015_20210717T101315_20210718T145224_0179_074_122_1980_MAR_O_NT_002.SEN3')

Next, we'll create another variable that takes this path, and finds and adds on the names of the manifest file within the SAFE folder. For Sentinel-3, this manifest is always called **xfdumanifest.xml** and contains very useful information about the nature of the contents of the SAFE format product.

In [3]:
# selecting SAFE manifest
SAFE_manifest = glob.glob(os.path.join(SAFE_directory, 'xfd*.xml'))[0]
display(Markdown('**Manifest file:** {}'.format(os.path.basename(SAFE_manifest))))

**Manifest file:** xfdumanifest.xml

Then, we'll read the manifest file in to a Python object format called a dictionary. This will make it easier for us to interact with the product in the next step.

In [4]:
# Reading SAFE manifest and outputting/collecting component names
tree = ET.parse(SAFE_manifest)
root = tree.getroot()
items = root.find('dataObjectSection')
SAFE_components = []
for item in items:
    display(Markdown("**File name:** {}".format(item[0][0].get('href'))))
    display(Markdown("*File info: {}*".format(item[0][0].get('textInfo'))))
    SAFE_components.append(os.path.join(SAFE_directory, item[0][0].get('href')))

**File name:** ./Oa01_radiance.nc

*File info: TOA radiance for OLCI acquisition band Oa01*

**File name:** ./Oa02_radiance.nc

*File info: TOA radiance for OLCI acquisition band Oa02*

**File name:** ./Oa03_radiance.nc

...

*File info: Tie-Point Geometries Annotations*

**File name:** ./tie_meteo.nc

*File info: Tie-Point Meteo Annotations*

**File name:** ./time_coordinates.nc

*File info: Time Coordinates Annotations*

The next section of code will display each of the components within the product. Scrolling through you can see the main groups of variables, including some you may already know or be interested in working with.

In [5]:
# Display component structure and variables
for SAFE_component in sorted(SAFE_components):
    ds = xr.open_dataset(SAFE_component)
    display(Markdown('**Filename:** {}'.format(os.path.basename(SAFE_component))))
    display(ds)
    ds.close()

**Filename:** Oa01_radiance.nc

<xarray.Dataset>
Dimensions:        (rows: 4090, columns: 4865)
Dimensions without coordinates: rows, columns
Data variables:
    Oa01_radiance  (rows, columns) float32 ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T14:52:24Z
    ...                     ...
    references:             S3IPF PDS 004.1 - i2r4 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-1-EO 06.11
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 1b Product, Radiance Oa01 Data Set

**Filename:** Oa02_radiance.nc

<xarray.Dataset>
Dimensions:        (rows: 4090, columns: 4865)
Dimensions without coordinates: rows, columns
Data variables:
    Oa02_radiance  (rows, columns) float32 ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T14:52:24Z
    ...                     ...
    references:             S3IPF PDS 004.1 - i2r4 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-1-EO 06.11
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 1b Product, Radiance Oa02 Data Set

**Filename:** Oa03_radiance.nc

...

<xarray.Dataset>
Dimensions:  (tie_rows: 4090, tie_columns: 77)
Dimensions without coordinates: tie_rows, tie_columns
Data variables:
    OAA      (tie_rows, tie_columns) float64 ...
    OZA      (tie_rows, tie_columns) float64 ...
    SAA      (tie_rows, tie_columns) float64 ...
    SZA      (tie_rows, tie_columns) float64 ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T14:52:24Z
    ...                     ...
    references:             S3IPF PDS 004.1 - i2r4 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-1-EO 06.11
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 1b Product, Tie-Point Geometries Data Set

**Filename:** tie_meteo.nc

<xarray.Dataset>
Dimensions:                          (tie_rows: 4090, tie_columns: 77,
                                      tie_pressure_levels: 25, wind_vectors: 2)
Dimensions without coordinates: tie_rows, tie_columns, tie_pressure_levels,
                                wind_vectors
Data variables:
    atmospheric_temperature_profile  (tie_rows, tie_columns, tie_pressure_levels) float32 ...
    horizontal_wind                  (tie_rows, tie_columns, wind_vectors) float32 ...
    humidity                         (tie_rows, tie_columns) float32 ...
    reference_pressure_level         (tie_pressure_levels) float32 ...
    sea_level_pressure               (tie_rows, tie_columns) float32 ...
    total_columnar_water_vapour      (tie_rows, tie_columns) float32 ...
    total_ozone                      (tie_rows, tie_columns) float32 ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T14:52:24Z
    ...                     ...
    references:             S3IPF PDS 004.1 - i2r4 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-1-EO 06.11
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 1b Product, Tie-Point Meteo Data Set

**Filename:** time_coordinates.nc

<xarray.Dataset>
Dimensions:     (rows: 4090)
Dimensions without coordinates: rows
Data variables:
    time_stamp  (rows) datetime64[ns] ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T14:52:24Z
    ...                     ...
    references:             S3IPF PDS 004.1 - i2r4 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-1-EO 06.11
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 1b Product, Time Stamps Data Set

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Querying OLCI file structure (Level-2)
[Back to top](#TOC_TOP)

</div>

Now let's look at the level-2 OLCI product that we downloaded.

In [6]:
# selecting SAFE directory
SAFE_directory = os.path.join(os.getcwd(), 'products', 
    'S3A_OL_2_WFR____20210717T101015_20210717T101315_20210718T221347_0179_074_122_1980_MAR_O_NT_003.SEN3')

Again, we'll create another variable that takes this path, and finds and adds on the names of the manifest file within the SAFE folder.  

In [7]:
# selecting SAFE manifest
SAFE_manifest = glob.glob(os.path.join(SAFE_directory, 'xfd*.xml'))[0]
display(Markdown('**Manifest file:** {}'.format(os.path.basename(SAFE_manifest))))

**Manifest file:** xfdumanifest.xml

Then, we'll read the manifest file for this level-2 data product in to another dictionary. 

In [8]:
# Reading SAFE manifest and outputting/collecting component names
tree = ET.parse(SAFE_manifest)
root = tree.getroot()
items = root.find('dataObjectSection')
SAFE_components = []
for item in items:
    display(Markdown("**File name:** {}".format(item[0][0].get('href'))))
    display(Markdown("*File info: {}*".format(item[0][0].get('textInfo'))))
    SAFE_components.append(os.path.join(SAFE_directory, item[0][0].get('href')))

**File name:** ./Oa01_reflectance.nc

*File info: Reflectance for OLCI acquisition band Oa01*

**File name:** ./Oa02_reflectance.nc

*File info: Reflectance for OLCI acquisition band Oa02*

**File name:** ./Oa03_reflectance.nc

...

*File info: Total suspended matter concentration*

**File name:** ./w_aer.nc

*File info: Aerosol Over Water*

**File name:** ./wqsf.nc

*File info: Water Quality and Science Flags*

Finally, let's display the components of the product...

In [9]:
# Display component structure and variables
for SAFE_component in sorted(SAFE_components):
    ds = xr.open_dataset(SAFE_component)
    display(Markdown('**Filename:** {}'.format(os.path.basename(SAFE_component))))
    display(ds)
    ds.close()

**Filename:** Oa01_reflectance.nc

<xarray.Dataset>
Dimensions:               (rows: 4090, columns: 4865)
Dimensions without coordinates: rows, columns
Data variables:
    Oa01_reflectance      (rows, columns) float32 ...
    Oa01_reflectance_err  (rows, columns) float32 ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T22:13:47Z
    ...                     ...
    references:             S3IPF PDS 004.3 - i2r3 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-2 07.01
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 2 WATER Product, Reflectance Oa01 Data...

**Filename:** Oa02_reflectance.nc

<xarray.Dataset>
Dimensions:               (rows: 4090, columns: 4865)
Dimensions without coordinates: rows, columns
Data variables:
    Oa02_reflectance      (rows, columns) float32 ...
    Oa02_reflectance_err  (rows, columns) float32 ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T22:13:47Z
    ...                     ...
    references:             S3IPF PDS 004.3 - i2r3 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-2 07.01
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 2 WATER Product, Reflectance Oa02 Data...

**Filename:** Oa03_reflectance.nc

...

<xarray.Dataset>
Dimensions:     (rows: 4090, columns: 4865)
Dimensions without coordinates: rows, columns
Data variables:
    TSM_NN      (rows, columns) float32 ...
    TSM_NN_err  (rows, columns) float32 ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T22:13:47Z
    ...                     ...
    references:             S3IPF PDS 004.3 - i2r3 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-2 07.01
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 2 WATER Product, Neural Net Total Susp...

**Filename:** w_aer.nc

<xarray.Dataset>
Dimensions:   (rows: 4090, columns: 4865)
Dimensions without coordinates: rows, columns
Data variables:
    A865      (rows, columns) float32 ...
    A865_err  (rows, columns) float32 ...
    T865      (rows, columns) float32 ...
    T865_err  (rows, columns) float32 ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T22:13:47Z
    ...                     ...
    references:             S3IPF PDS 004.3 - i2r3 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-2 07.01
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 2 WATER Product, Aerosol Data Set

**Filename:** wqsf.nc

<xarray.Dataset>
Dimensions:  (rows: 4090, columns: 4865)
Dimensions without coordinates: rows, columns
Data variables:
    WQSF     (rows, columns) uint64 ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T22:13:47Z
    ...                     ...
    references:             S3IPF PDS 004.3 - i2r3 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-2 07.01
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 2 WATER Product, Classification, Quali...

<div class="alert alert-danger" role="alert">

## <a id='section3'></a>3. Applying your knowledge
[Back to top](#TOC_TOP)

</div>

<div class="alert alert-block alert-warning">

### Challenge:

What are differences you see between the level-1B and level-2 files? <div>

**Answer:** In level 1B, it is mostly radiance and in level 2, it is reflectance

### Enter your solution here
* ...
* ...

* At level-1B, OLCI products contain radiance data. At level-2 they contain reflectance data.
* At level-1B, there are 21 bands. At level 2, bands 13/14/15/19/20 are missing, as these are used for atmospheric characterisation/correction only.
* At level-2, we have geophysical products, e.g. chlorophyll concentration, total suspended matter.
* The flags are different at level-1B and level-2.

For more information on OLCI data file contents, check out the 
<a href="https://eumetsatspace.atlassian.net/wiki/spaces/SEN3/pages/1597800573/OLCI+products+and+file+types" target="_blank">OLCI product pages in the Sentinel-3 knowledge base</a>.


https://documentation.dataspace.copernicus.eu/Data/SentinelMissions/Sentinel3.html

<hr>
<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1a_OLCI_data_access_Data_Store.ipynb" target="_blank"><< Accessing OLCI data</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_3_OLCI_coverage.ipynb" target="_blank">Determining OLCI product coverage >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>